In [49]:
import pandas as pd
from IPython.display import display
import seaborn as sns
from matplotlib import pyplot as plt
import nltk
import re
from os import path
from unicodedata import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
%matplotlib inline

As técnicas utilizadas no protótipo do Dashboard de análise de gastos relacionados ao ODS 11 utiliza técnicas de processamento de linguagem natural, porém em nível muito básico. O principal obstáculo para essa prática é a aplicação de aprendizado supervisionado, já que não temos uma classificação pronta para cada ODS. No entanto, podemos tentar melhorar esse algoritmo utilizando as categorias criadas a partir das listas de palavras do dashboard, e a partir delas criar um aprendizado para analisar novos dados de maneira mais profunda com NLP.

Exemplificaremos esse processo com uma cidade, Campinas. O modelo criado poderá ser utilizado para novos dados gerados dinâmicamente, como por exemplo, dados provenientes de APIs, podendo classificá-los como relacionados ao ODS ou não.

In [22]:
df_csv = pd.read_csv(f'../data/despesas/by_city/despesas-2019-campinas.csv', sep=",")

In [23]:
df_csv['vl_despesa'] = df_csv['vl_despesa'].apply(lambda x: float(x.replace(",", "."))).astype(float)

In [24]:
len(df_csv)

101196

In [25]:
df = df_csv.copy()
df['historico_despesa'] = df['historico_despesa'].apply(lambda x: re.sub(r'\W+', ' ', x))
df['historico_despesa'] = df['historico_despesa'].apply(lambda x: re.sub(r'^[a-zA-Z]+$', ' ', x))
txt = df.historico_despesa.str.lower().str.cat(sep=' ')
words = nltk.tokenize.word_tokenize(txt)
word_dist = nltk.FreqDist(words)
stopwords = nltk.corpus.stopwords.words('portuguese')
words_except_stop_dist = nltk.FreqDist(w for w in words if w not in stopwords) 
rslt = pd.DataFrame(words_except_stop_dist.most_common(101196),
                    columns=['Word', 'Frequency'])
rslt = rslt[rslt.Word.str.isalpha()]

In [26]:
rslt.head()

,Word,Frequency
2,nº,21498
3,prot,20631
5,empenho,18769
6,pmc,16227
7,pagamento,15047


Classificaremos se a despesa está relacionada ao ODS ou não.

In [27]:
ods_11_1 = [
    "habitação",
    "habitacao",
    "habitacional",
    "favelas",
    "urbano",
    "urbanização",
    "urbanismo",
    "urbanizacao",
    "favela",
    "básicos",
]
ods_11_2 = [
    "transporte",
    "metrô",
    "metro",
    "onibus",
    "ônibus",
    "trem",
    "vlt",
    "monotrilho",
    "bonde",
    "frota",
    "trens",
    "transportes",
]
ods_11_3 = [
    "sustentável",
    "sustentavel",
    "sustentabilidade",
    "participativo",
    "participativa",
    "assentamentos",
]
ods_11_4 = [
    "partimônio",
    "cultura",
    "restauração",
    "patrimonio",
    "restauracao",
    "reserva",
    "natural",
    "natureza",
    "cultural",
    "patrimonial",
    "meio-ambiente",
]
ods_11_5 = [
    "mortalidade",
    "catástrofe",
    "catastrofe",
    "catástrofes",
    "catastrofes",
    "acidente",
    "acidentes",
    "enchentes",
    "enchente",
    "deslizamento",
    "desastre",
    "desastres",
    "chuva",
    "chuvas",
    "barragem",
    "contaminação",
    "despoluição",
    "contaminacao",
    "despoulicao",
    "contaminado",
    "contaminada",
]
ods_11_6 = [
    "ar",
    "poluição",
    "resíduos",
    "poluicao",
    "residuos",
    "esgoto",
    "esgotos",
    "saneamento",
    "coleta",
    "lixo",
    "lixão",
    "lixao",
    "lixoes",
    "lixões",
    "aterro",
    "aterros",
    "reciclagem",
    "reciclado",
    "recicla",
    "descarte",
]
ods_11_7 = [
    "espaço",
    "espaco",
    "acessível",
    "acessibilidade",
    "convivência",
    "convivencia",
    "conviver",
    "acessivel",
    "cadeirante",
    "cego",
    "deficiente",
    "deficientes",
    "deficiencia",
    "deficiência",
    "idoso",
    "idosos",
    "idosa",
    "mulher",
    "mulheres",
    "verde",
    "verdes",
]
ods_11_a = ["rural", "campo", "periurbana"]
ods_11_b = [
    "eficiência",
    "eficiencia",
    "clima",
    "climático",
    "climatico",
    "aquecimento",
    "sendai",
]
ods_11_c = []

ods_11 = (
    ods_11_1
    + ods_11_2
    + ods_11_3
    + ods_11_4
    + ods_11_5
    + ods_11_6
    + ods_11_7
    + ods_11_a
    + ods_11_b
    + ods_11_c
)

In [28]:
rslt["ods"] = rslt["Word"].apply(lambda x: x in ods_11)

In [32]:
rslt.head()

,Word,Frequency,ods
2,nº,21498,False
3,prot,20631,False
5,empenho,18769,False
6,pmc,16227,False
7,pagamento,15047,False


In [34]:
ods_rslt = rslt[rslt.ods]

In [35]:
ods_list = list(ods_rslt["Word"])

In [37]:
def is_ods(txt, ods_list):
    is_ods = False
    for ods in ods_list:
        if ods in txt:
            is_ods = True
    return is_ods

In [36]:
df_csv["is_ods"] = df_csv["historico_despesa"].apply(lambda x: is_ods(x, ods_list))

In [39]:
df_csv[['historico_despesa', 'is_ods']].head()

,historico_despesa,is_ods
0,1 PRESTACAO DE SERVICOS DE TELECOMUNICACOES A...,False
1,1 ADIANTAMENTO DE FOLHA DE FUNCIONARIOS EFETIV...,False
2,1 PEDIDO DE ADIANTAMENTO PARA PRONTO PAGAMENT...,False
3,1 LOCACAO DE SOLUCAO MOVEL E PORTATIL DE TELEJ...,False
4,EMPENHO REF. REGISTRO DE PREÇOS DE ÁGUA MINERA...,False


Agora temos um DataFrame com labels que indicam se o gasto está relacionado ou não, e podemos aplicar um aprendizado supervisionado para analisar, usando técnicas mais profundas de NLP (como por exemplo, n-gramas) para prever essa mesma relação para gastos futuros.

In [54]:
X_train, X_test, y_train, y_test = train_test_split(df_csv['historico_despesa'],
                                                    pd.get_dummies(df_csv['is_ods']), 
                                                    random_state=1997)

Aplicamos count vectorizer para tokens alfanuméricos, e usando n-gramas para até n=2, em seguida aplicando um classificador a partir de uma regressão logística.

In [55]:
pl = Pipeline([
        ('vec', CountVectorizer(token_pattern='[A-Za-z0-9]+(?=\\s+)', ngram_range=(1,2))),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

In [56]:
pl.fit(X_train, y_train)

/home/felipecgonc/.virtualenvs/deephack/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('vec',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='[A-Za-z0-9]+(?=\\s+)',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(C=1.0,
                                                                  class_weight=None,
                                                                  dual=False,
                                                                  fit_int

In [57]:
accuracy = pl.score(X_test, y_test)

In [58]:
accuracy

0.9956915293094589

Vemos que a precisão é boa. No entanto, o treinamento do modelo se baseia nas categorias aplicadas automáticamente por uma busca simples de palavras. Caso as despesas fossem categorizadas manualmente, poderíamos extrair um aprendizado menos óbvio desse modelo. Sendo assim, o exemplo aqui apresentado tem suas limitações, mas a metodologia pode ser aplicada a um modelo com dados melhores.

Tal modelo pode ser aplicado a novos dados vindos da API, permitindo classificar instantâneamente se estes estão relacionados ao ODS 11 ou não.